In [223]:
import pandas as pd
import numpy as np
import json

## DATA

In [224]:
session_data_raw = pd.read_csv(filepath_or_buffer="data/Session_data.csv")
signup_data_raw = pd.read_csv(filepath_or_buffer="data/Signup_data.csv")

with open("data/ltv.json", "r") as json_file:
    ltv_data = json.load(json_file)

### Initial look at data

In [225]:
session_data_raw[:10]

,Session ID,User ID,Market,Sign-up,Experiment ID,Variant ID
0,7BIO49O640BIASIBALZJ,004YOVNZC1NW68QLXU,Norway,False,Experiment032,Control
1,WLD3L4N54AUGPRUHUJG9,007TH913EDSVIDR248,Finland,False,Experiment037,Variant 1
2,3ALVA866B3LL9RNDNLCY,00B1YBLAAG6DEIFCOH,Denmark,False,NaN,NaN
3,H638ZR9TECATB7QRTP1O,00L73RO9N9YQW4TLFG,Denmark,True,Experiment037,Variant 1
4,WNKHTTGTYZVBPJO2KT07,00QYV1X2MN5MBBKTZ9,Norway,False,Experiment037,Control
5,KLHKJ2O7AF207MRNU549,00RDZ0X9037G65ARSC,Sweden,False,Experiment037,Variant 2
6,JYBY694NAPC8FYM7HU7D,00U5F18PAZLFL54IZM,Sweden,False,Experiment037,Variant 1
7,X9EO00FDBDUO888MMY56,00V1ZT63REHRX5LOC8,Norway,True,Experiment037,Variant 2
8,CA10G00GLY79NUJG2M5G,00VRKG3QBA3K69CEJQ,Norway,False,Experiment037,Variant 1
9,CHIRLT84GOA0I4ECW13I,00X9DQBSH6ADZSURQ2,Sweden,False,Experiment037,Variant 1


In [226]:
signup_data_raw[:10]

,Session ID,Sign-up
0,0HYQ2HO0GC2XCTOY3NGQ,advanced
1,7X5AHWLLOSZ2VUT2DG11,advanced
2,I0ZS1QTW56E7KC6KPR3J,advanced
3,S2G1I1F2QPW4CUZ9VLMK,advanced
4,BK1RMVLBGZZXLPWI5OME,advanced
5,G90BYI0AWJYCFFBRWBLV,advanced
6,R46RPPGNY5VP2PLQ0YMH,advanced
7,M6EN0WNDSD3S44IMNJ4W,advanced
8,BP4FUZS1HNXZ45BYBA9L,advanced
9,WL2O2UONPOD0W2IGDEEK,advanced


In [227]:
ltv_data

{'dropout': 0, 'free': 3120, 'basic': 5280, 'advanced': 21800}

### Data Transformations

In [228]:
# Filtering off irrelevant experiments
session_data = session_data_raw[session_data_raw['Experiment ID'] == "Experiment037"]

In [229]:
# Joining two tables
all_data = pd.merge(
    left = session_data,
    right = signup_data_raw,
    how = 'left',
    on = 'Session ID',
    suffixes=('_session', '_signup')
)

all_data.rename(columns={'Sign-up_session': 'Sign-up', 'Sign-up_signup': 'Subscription'}, inplace = True)
all_data['Subscription'].fillna("dropout", inplace=True)

market_mappings = {
    "Denmark": "Denmark",
    "Sweden": "Sweden",
    "Finland": "Finland",
    "Norway": "Norway",
    "Suomi": "Finland",
    "Sverige": "Sweden",
    "swe": "Sweden",
    "danmark": "Denmark",
    "denmark": "Denmark"
}

for key in market_mappings.keys():
    all_data['Market'] = all_data['Market'].replace(key, market_mappings[key])

grouped_data = all_data.groupby(by = ['Market', 'Variant ID', 'Sign-up', 'Subscription']) \
                       .agg(session_count = ('Session ID', 'nunique')) \
                       .sort_values(by = 'session_count', ascending = False) \
                       .reset_index()

def add_ltv(row):
    return row['session_count'] * ltv_data[row['Subscription']]

grouped_data['ltv'] = grouped_data.apply(add_ltv, axis = 1)

grouped_data.to_csv('processed/grouped_data.csv')
grouped_data

,Market,Variant ID,Sign-up,Subscription,session_count,ltv
0,Denmark,Control,False,dropout,2288,0
1,Denmark,Variant 2,False,dropout,2216,0
2,Denmark,Variant 1,False,dropout,2185,0
3,Sweden,Variant 2,False,dropout,2051,0
4,Sweden,Control,False,dropout,2045,0
5,Sweden,Variant 1,False,dropout,1992,0
6,Finland,Control,False,dropout,1569,0
7,Finland,Variant 2,False,dropout,1411,0
8,Finland,Variant 1,False,dropout,1401,0
9,Norway,Control,False,dropout,1300,0


### Data Exploration

In [230]:
all_data \
    .groupby(by = 'Variant ID') \
    .agg(
        session_count=('Session ID', 'nunique')
    ) \
    .sort_values(by = 'session_count', ascending = False) \
    .reset_index()

,Variant ID,session_count
0,Control,8434
1,Variant 2,8050
2,Variant 1,7988


### Data Analysis

In [231]:
# Set the markets filter
markets = [
    'Finland', 
    'Norway', 
    'Sweden', 
    'Denmark'
]

### Signups and dropouts!

In [232]:
# Total sessions broken into Variants
total_sessions = grouped_data[grouped_data['Market'].isin(markets)] \
                .groupby(by = 'Variant ID') \
                .agg(total_sessions = ('session_count', 'sum')) \
                .sort_values(by = 'total_sessions', ascending = False) \
                .reset_index()

# Total dropouts broken into Variants
dropouts = grouped_data[(grouped_data['Market'].isin(markets)) & (grouped_data['Sign-up'] == False)] \
                .groupby(by = 'Variant ID') \
                .agg(dropout_count = ('session_count', 'sum')) \
                .sort_values(by = 'dropout_count', ascending = False) \
                .reset_index()

# Total signups broken into Variants
signups = grouped_data[(grouped_data['Market'].isin(markets)) & (grouped_data['Sign-up'] == True)] \
                .groupby(by = 'Variant ID') \
                .agg(signup_count = ('session_count', 'sum')) \
                .sort_values(by = 'signup_count', ascending = False) \
                .reset_index()

signups_all = pd.merge(
        left = pd.merge(
                left = total_sessions, 
                right = signups, 
                how = 'inner', 
                on = 'Variant ID'
        ),
        right = dropouts,
        how = 'inner',
        on = 'Variant ID'
)

signups_all['signup_rate'] = ( signups_all['signup_count'] * 100 ) / signups_all['total_sessions']
signups_all['dropout_rate'] = ( signups_all['dropout_count'] * 100 ) / signups_all['total_sessions']
signups_all.sort_values(by = 'dropout_rate')

,Variant ID,total_sessions,signup_count,dropout_count,signup_rate,dropout_rate
2,Variant 1,7988,1180,6808,14.772158,85.227842
1,Variant 2,8050,1179,6871,14.645963,85.354037
0,Control,8434,1232,7202,14.607541,85.392459


### Subscriptions

In [233]:
# Set the subscription_type filter
subscription_type = [
    'free',
    'basic',
    'advanced'
]

In [234]:
signed_up_sessions = grouped_data[(grouped_data['Market'].isin(markets)) & (grouped_data['Subscription'].isin(subscription_type))]\
                            .groupby(by = ['Market', 'Variant ID', 'Subscription']).agg(total_sessions=('session_count', 'sum'), LTV=('ltv', 'sum')).reset_index()\
                            .sort_values(by = ['Market', 'Variant ID'])

signed_up_sessions

,Market,Variant ID,Subscription,total_sessions,LTV
0,Denmark,Control,advanced,44,959200
1,Denmark,Control,basic,97,512160
2,Denmark,Control,free,241,751920
3,Denmark,Variant 1,advanced,54,1177200
4,Denmark,Variant 1,basic,69,364320
5,Denmark,Variant 1,free,256,798720
6,Denmark,Variant 2,advanced,36,784800
7,Denmark,Variant 2,basic,78,411840
8,Denmark,Variant 2,free,238,742560
9,Finland,Control,advanced,29,632200


### Market Evaluation

In [238]:
temp_eval = signed_up_sessions.groupby(by = ['Market', 'Variant ID']).agg(total_sessions=('total_sessions', 'sum'), total_ltv=('LTV', 'sum'))\
                  .assign(ltv_per_subscription=lambda x: round(x['total_ltv'] / x['total_sessions']))\
                  .sort_values(by = ['Market', 'ltv_per_subscription'], ascending = False)\
                  .reset_index()

market_eval = pd.DataFrame()

for market in markets:
    temp = temp_eval.loc[temp_eval['Market'] == market].copy()
    control_value = temp.loc[temp['Variant ID'] == 'Control', 'ltv_per_subscription'].values[0]
    temp.loc[:, 'control_value'] = control_value
    temp.loc[:, 'variation'] = (temp['ltv_per_subscription'] - control_value) / control_value
    market_eval = pd.concat([market_eval, temp], ignore_index=True)
  
market_eval.to_csv('processed/final.csv')
market_eval[market_eval['Variant ID'] != 'Control']

,Market,Variant ID,total_sessions,total_ltv,ltv_per_subscription,control_value,variation
1,Finland,Variant 1,231,1300040,5628.0,5749.0,-0.021047
2,Finland,Variant 2,237,1306560,5513.0,5749.0,-0.041051
4,Norway,Variant 1,210,1201480,5721.0,5787.0,-0.011405
5,Norway,Variant 2,232,1246480,5373.0,5787.0,-0.071540
6,Sweden,Variant 1,360,2001520,5560.0,4897.0,0.135389
7,Sweden,Variant 2,358,1987400,5551.0,4897.0,0.133551
9,Denmark,Variant 1,379,2340240,6175.0,5820.0,0.060997
11,Denmark,Variant 2,352,1939200,5509.0,5820.0,-0.053436


### Overal Evaluation

In [236]:
overall_eval = signed_up_sessions.groupby(by = ['Variant ID']).agg(total_sessions=('total_sessions', 'sum'), total_ltv=('LTV', 'sum'))\
                  .assign(ltv_per_subscription=lambda x: round(x['total_ltv'] / x['total_sessions']))\
                  .sort_values(by = ['ltv_per_subscription'], ascending = False)\
                  .reset_index()

overall_eval['control_value'] = overall_eval.loc[overall_eval['Variant ID'] == 'Control']['ltv_per_subscription'].values[0]
overall_eval['variation'] = (overall_eval['ltv_per_subscription'] - overall_eval['control_value']) / overall_eval['control_value']
overall_eval

,Variant ID,total_sessions,total_ltv,ltv_per_subscription,control_value,variation
0,Variant 1,1180,6843280,5799.0,5534.0,0.047886
1,Control,1232,6817440,5534.0,5534.0,0.000000
2,Variant 2,1179,6479640,5496.0,5534.0,-0.006867
